In [1]:
import cobra
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal

In [2]:
model = cobra.io.read_sbml_model("Arnold2014_WT_Cntl.xml")
samples_control = pd.read_csv('ResultsCHRR_ANmodel_Col0_Control_100000.csv',header=None)
samples_cold = pd.read_csv('ResultsCHRR_ANmodel_Col0_Cold_100000.csv',header=None)
samples_fum_control = pd.read_csv('ResultsCHRR_ANmodel_Fum2_Control_100000.csv',header=None)
samples_fum_cold = pd.read_csv('ResultsCHRR_ANmodel_Fum2_Cold_100000.csv',header=None)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f885214ead0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f881c01de10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f8855aed8d0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

In [39]:
# Comparing the reactions which are required vs the reactions that are not required in cold vs control
print(samples_control.shape[1])
req_both = [] #reactions required in both conditions 
req_cold = [] #required only in the cold 
for i in range(samples_control.shape[1]): #iterating through all reactions 
    x = pd.Series.tolist(samples_cold.iloc[:,i])
    y = pd.Series.tolist(samples_control.iloc[:,i])
#     if model.reactions[i].id in ["FrcK_c","GlyDH_m","AMT_m","DHLDH1_m","GlyHMT_m","P5CDHNADP_m","NGAM_c","NGAM_m"]:
#     #["FBPase_h","F6PK1_c","FumHA_c","Asnase_c","Tr_DIC1","Tr_DIC2","Si_H"]:
#         print(model.reactions[i].id)
#         print(model.reactions[i].reaction)
#         print("Cold: {},{}".format(min(x),max(x)))
#         print("Control: {},{} \n".format(min(y),max(y)))
    if round(min(x),4)>0.0 and round(min(y),4)>0.0:
        req_both.append(i)
    if round(max(x),4)<0.0 and round(max(y),4)<0.0:
        req_both.append(i)
    if round(min(x),4)>0.0 and round(min(y),4)<=0.0:
        req_cold.append(i)
        print(i)
        print(model.reactions[i].id)
        print(model.reactions[i].reaction)
        print("Cold: {},{}".format(min(x),max(x)))
        print("Control: {},{} \n".format(min(y),max(y)))
    if round(max(x),3)<0.0 and round(max(y),3)>=0.0:
        req_cold.append(i)
        mutant_req.append(i)
        print(i)
        print(model.reactions[i].id)
        print(model.reactions[i].reaction)
        print("Cold: {},{}".format(min(x),max(x)))
        print("Control: {},{} \n".format(min(y),max(y)))
print(len(req_both))
print(len(req_cold))

# Reactions that are required in both conditions in Col0 and not in Fum2 and vice versa
req_col = []
req_fum = []
for i in range(samples_control.shape[1]):
    x = pd.Series.tolist(samples_cold.iloc[:,i])
    y = pd.Series.tolist(samples_control.iloc[:,i])
    xm = pd.Series.tolist(samples_fum_cold.iloc[:,i])
    ym = pd.Series.tolist(samples_fum_control.iloc[:,i])
    if round(min(x),4)>0.0 and round(min(y),4)>0.0:
        req_col.append(i)
    if round(max(x),4)<0.0 and round(max(y),4)<0.0:
        req_col.append(i)
    if round(min(xm),4)>0.0 and round(min(ym),4)>0.0:
        req_fum.append(i)
    if round(max(xm),4)<0.0 and round(max(ym),4)<0.0:
        req_fum.append(i)

print("--------")
col_only = list(set(req_col) - set(req_fum))
for i in col_only:
    print(model.reactions[i].id)
    print(model.reactions[i].reaction)
print("--------")
fum_only = list(set(req_fum) - set(req_col))
for i in fum_only:
    print(model.reactions[i].id)
    print(model.reactions[i].reaction)

552
10
FBPase_h
FBP_h + H2O_h --> F6P_h + Pi_h
Cold: 0.0012149,80.367
Control: 1.7794e-05,79.732 

59
F6PK1_c
ATP_c + F6P_c --> ADP_c + FBP_c + H_c
Cold: 0.00029952,83.309
Control: 1.7413e-05,88.953 

164
FumHA_c
Mal_c --> Fum_c + H2O_c
Cold: 1.3345,4.494
Control: 6.9924e-06,1.1875 

170
Asnase_c
Asn_c + H2O_c --> Asp_c + NH4_c
Cold: 0.00010108,30.04
Control: 2.8301e-05,26.005 

339
Tr_DIC1
Mal_c + Pi_m <=> Mal_m + Pi_c
Cold: -999.99,-700.83
Control: -999.65,1000.0 

340
Tr_DIC2
Pi_m + SCA_c <=> Pi_c + SCA_m
Cold: 700.83,999.99
Control: -1000.0,999.65 

381
Tr_H2Op
H2O_c <=> H2O_p
Cold: -2.5127,-0.00071728
Control: -2.4302,-0.00048091 

444
Si_H
 <=> H_c
Cold: -18.079,-5.9417
Control: -26.319,0.13982 

225
8
--------
GluSeADH_m
GluP_m + H_m + NADPH_m --> Glu_DASH_SeA_m + NADP_m + Pi_m
ACoAC_h
ATP_h + A_DASH_CoA_h + HCO3_h --> ADP_h + H_h + M_DASH_CoA_h + Pi_h
ACPM_h
ACP_h + M_DASH_CoA_h --> CoA_h + M_DASH_ACP_h
ForDH_h
CO2_h + NADH_h <=> For_h + NAD_h
ForTHFL_h
ATP_h + For_h + H_h + TH

In [ ]:
# Saving all distribution plots to a PDF 
start = time.time()
pdf = matplotlib.backends.backend_pdf.PdfPages("Fum2Comparison_All.pdf")
totaldiff = []
for i in range(samples_control.shape[1]): #iterating through all reactions 
    x = pd.Series.tolist(samples_fum_cold.iloc[:,i])
    y = pd.Series.tolist(samples_fum_control.iloc[:,i])
    mins = min(min(x),min(y))
    maxs = max(max(x),max(y))
    p=0.1
#     if i in [136,137,138,139]:
#         p=0.1
#     else:
#         stat, p = kruskal(x, y)
    if p>0.001:
#         print("Samples are from the same distribution")
        bns = 100
        weights_x = np.ones_like(x)/float(len(x))
        weights_y = np.ones_like(y)/float(len(y))
        xn, xbins, xpatches = plt.hist(x, bins=bns, alpha=0.5, weights=weights_x, color = "blue", lw=0)
        yn, ybins, ypatches = plt.hist(y, bins=bns, alpha=0.5, weights=weights_y, color = "red", lw=0)
        stat, p = mannwhitneyu(xn, yn)
        plt.xlabel('Flux')
        plt.ylabel('Frequency')
        gs = model.reactions[i].genes
        gene_names = []
        for g in gs:
            gene_names.append(g.id)
        if model.reactions[i].id in constr:
            plt.title('{} : {} \n{}'.format(i,model.reactions[i].reaction,gene_names),color="red",fontsize=8)
        else:
            plt.title('{} : {} \n{}'.format(i,model.reactions[i].reaction,gene_names),fontsize=6)
        pdf.savefig()
        plt.close()
        pass
    else: 
        bns = 100
        weights_x = np.ones_like(x)/float(len(x))
        weights_y = np.ones_like(y)/float(len(y))
        xn, xbins, xpatches = plt.hist(x, bins=bns, alpha=0.5, weights=weights_x, color = "blue", lw=0)
        yn, ybins, ypatches = plt.hist(y, bins=bns, alpha=0.5, weights=weights_y, color = "red", lw=0)
        stat, p = mannwhitneyu(xn, yn)
        plt.xlabel('Flux')
        plt.ylabel('Frequency')
        plt.title('**{} : {}**'.format(i,model.reactions[i].reaction))
        pdf.savefig()
        plt.close()
        totaldiff.append(i)
pdf.close()
end = time.time()
print(end-start) 
print(len(totaldiff))

In [ ]:
totaldiff = []
pdf = matplotlib.backends.backend_pdf.PdfPages("AllComparisons_All.pdf")
for i in range(samples_control.shape[1]): #iterate through all reactions 
    col0 = pd.Series.tolist(samples_control.iloc[:,i])
    col0_cold = pd.Series.tolist(samples_cold.iloc[:,i])
    fum2 = pd.Series.tolist(samples_fum_control.iloc[:,i])
    fum2_cold = pd.Series.tolist(samples_fum_cold.iloc[:,i])
    stat, p = kruskal(col0,col0_cold,fum2,fum2_cold)
    if p<0.001: #Then samples are from different distributions 
        totaldiff.append(i)
        bns = 100
        weights_x = np.ones_like(col0)/float(len(col0))
        weights_y = np.ones_like(col0_cold)/float(len(col0_cold))
        weights_x_fum = np.ones_like(fum2)/float(len(fum2))
        weights_y_fum = np.ones_like(fum2_cold)/float(len(fum2_cold))
        xn, xbins, xpatches = plt.hist(col0, bins=bns, alpha=0.2, weights=weights_x, color = "red", lw=0)
        yn, ybins, ypatches = plt.hist(col0_cold, bins=bns, alpha=0.2, weights=weights_y, color = "blue", lw=0)
        xn, xbins, xpatches = plt.hist(fum2, bins=bns, alpha=0.2, weights=weights_x, color = "pink", lw=0)
        yn, ybins, ypatches = plt.hist(fum2_cold, bins=bns, alpha=0.2, weights=weights_y, color = "cyan", lw=0)
        plt.xlabel('Flux')
        plt.ylabel('Frequency')
        gs = model.reactions[i].genes
        gene_names = []
        for g in gs:
            gene_names.append(g.id)
        if model.reactions[i].id in constr:
            plt.title('{} : {} \n{}'.format(i,model.reactions[i].reaction,gene_names),color="red",fontsize=8)
        else:
            plt.title('{} : {} \n{}'.format(i,model.reactions[i].reaction,gene_names),fontsize=6)
        pdf.savefig()
        plt.close()
pdf.close()
print(len(totaldiff))
print(samples_control.shape)